In [2]:
# Importar bibliotecas

import pandas as pd
import pulp

In [ ]:
# Crear/leer datos

def leer_datos() -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.Timestamp, int]:
    """Crea DataFrames con los datos de los proyectos, periodos y muelles; y la fecha inicial.

    Returns
    -------
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    muelles : pd.DataFrame
        DataFrame con las dimensiones de los muelles.
    fecha_inicial : str
        Fecha inicial del primer periodo de los proyectos en formato 'YYYY-MM-DD'. 
    MOVED_PROJECTS_PENALTY_PER_MOVEMENT : int
        Penalización por cada movimiento de un barco a otro muelle en un periodo.
    """    

    # Aquí lógica para leer los datos desde un archivo o base de datos
    proyectos = pd.DataFrame({
        'eslora': [120, 100, 120],
        'manga': [18, 15, 18],
        'proyecto_id': ['PRO1', 'PRO2', 'PRO3'],
        'facturacion_diaria': [1000, 800, 1200],
        'proyecto_a_optimizar': [True, True, False]})
    
    proyectos.set_index('proyecto_id', inplace=True)

    periodos = pd.DataFrame({
        'tipo_desc': ['FLOTE', 'FLOTE', 'FLOTE','FLOTE','FLOTE'],
        'fecha_inicio': ['2025-08-08', '2025-08-21', '2025-08-10', '2025-08-17', '2025-08-17'],
        'fecha_fin': ['2025-08-20', '2025-08-26', '2025-08-16', '2025-08-19', '2025-08-25'],
        'nombre_area': ['SIN UBICACION ASIGNADA', 'SIN UBICACION ASIGNADA', 'SIN UBICACION ASIGNADA', 'SIN UBICACION ASIGNADA', 'MUELLE SUR'],
        'proyecto_id': ['PRO1', 'PRO1', 'PRO2', 'PRO2', 'PRO3'],
        'periodo_id': [0, 1, 0, 1, 0]})
    
    periodos['fecha_inicio'] = pd.to_datetime(periodos['fecha_inicio'])
    periodos['fecha_fin'] = pd.to_datetime(periodos['fecha_fin'])
    periodos['id_proyecto_reparacion'] = periodos['proyecto_id'] + '_' + periodos['periodo_id'].astype(str)
    periodos.set_index('id_proyecto_reparacion', inplace=True)

    muelles = pd.DataFrame({
        'longitud': [130, 110],
        'ancho': [20, 20],
        'nombre': ['MUELLE SUR', 'MUELLE NORTE']})

    muelles.set_index('nombre', inplace=True)

    fecha_inicial = periodos['fecha_inicio'].min()

    MOVED_PROJECTS_PENALTY_PER_MOVEMENT = 50

    return proyectos, periodos, muelles, fecha_inicial, MOVED_PROJECTS_PENALTY_PER_MOVEMENT



In [15]:
# Preprocesar datos

def preprocesar_datos(proyectos: pd.DataFrame, periodos: pd.DataFrame, muelles: pd.DataFrame, fecha_inicial: pd.Timestamp) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, list, set, set]:
    """Preprocesa los datos de proyectos, periodos y muelles para su uso en la optimización.

    Parameters
    ----------
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    muelles : pd.DataFrame  
        DataFrame con las dimensiones de los muelles.
    fecha_inicial : str
        Fecha inicial del primer periodo de los proyectos en formato 'YYYY-MM-DD'.
    
    Returns
    -------
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos, con fechas convertidas a enteros.
    muelles : pd.DataFrame
        DataFrame con las dimensiones de los muelles.
    dias : list
        Lista de días desde la fecha inicial hasta la fecha final de los periodos.
    set_a_optimizar : set
        Set de proyectos a optimizar.
    set_no_optimizar : set
        Set de proyectos que no optimizar.
    """

    # Convertir fechas a integer
    periodos['fecha_inicio'] = (periodos['fecha_inicio'] - fecha_inicial).dt.days
    periodos['fecha_fin'] = (periodos['fecha_fin'] - fecha_inicial).dt.days

    # Crear una lista de días
    dias = list(range(periodos['fecha_inicio'].min(), periodos['fecha_fin'].max()+1))

    # Crear set de proyectos confirmados y sin confirmar
    set_a_optimizar = set(proyectos[proyectos['proyecto_a_optimizar']].index)
    set_no_optimizar = set(proyectos[~proyectos['proyecto_a_optimizar']].index)

    # Columna de dias y localizaciones disponibles
    periodos['ubicaciones'] = periodos.apply(lambda row: row['nombre_area'] if row['nombre_area'] != 'SIN UBICACION ASIGNADA' 
                                             else [m for m in muelles.index if (muelles.loc[m, 'longitud'] >= proyectos.loc[row['proyecto_id'], 'eslora'] and 
                                                                                muelles.loc[m, 'ancho'] >= proyectos.loc[row['proyecto_id'], 'manga'])], axis=1)
    
    periodos['dias'] = periodos.apply(lambda row: list(range(row['fecha_inicio'], row['fecha_fin'] + 1)), axis=1)

    return proyectos, periodos, muelles, dias, set_a_optimizar, set_no_optimizar


In [16]:
# Definir variables de decisión

def definir_variables(periodos: pd.DataFrame, set_a_optimizar: set) -> tuple[dict, dict, dict]:
    """Define las variables de decisión del problema de optimización.

    Parameters
    ----------
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    set_a_optimizar : set
        Set de proyectos a optimizar.

    Returns
    -------
    x : dict
        Diccionario de variables binarias que indican si un periodo está asignado a un muelle en un día específico.
    y : dict
        Diccionario de variables binarias que indican si un periodo está asignado (1) o no (0).
    m : dict
        Diccionario de variables binarias que indican si un periodo se mueve de un muelle a otro en un día específico.
    """
    
    # Definir variables para cada periodo solo en los días y localizaciones correspondientes

    x = {(p_k, d, loc): pulp.LpVariable(f"x_{p_k}_{d}_{loc}",(p_k, d, loc), cat='Binary')
         for p_k in periodos[periodos["proyecto_id"].isin(set_a_optimizar)].index
         for d in periodos.loc[p_k, 'dias']
         for loc in periodos.loc[p_k, 'ubicaciones']}
    
    y = {p: pulp.LpVariable(f"y_{p}", p, cat='Binary')
         for p in set_a_optimizar}
    
    m = {(p_k, d): pulp.LpVariable(f"m_{p_k}_{d}", (p_k, d), cat='Binary')
         for p_k in periodos[periodos["proyecto_id"].isin(set_a_optimizar)].index if len(periodos.loc[p_k, 'ubicaciones']) > 1
         for d in periodos.loc[p_k, 'dias']}
    
    return x, y, m

In [17]:
def definir_funcion_objetivo(x: dict, m: dict, proyectos: pd.DataFrame, periodos: pd.DataFrame, set_a_optimizar: set, MOVED_PROJECTS_PENALTY_PER_MOVEMENT: int) -> pulp.LpAffineExpression:
    """Define la función objetivo del problema de optimización.

    Parameters
    ----------
    x : dict
        Diccionario de variables binarias que indican si un periodo está asignado a un muelle en un día específico.
    m : dict
        Diccionario de variables binarias que indican si un periodo se mueve de un muelle a otro en un día específico.
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    MOVED_PROJECTS_PENALTY_PER_MOVEMENT : int
        Penalización por cada movimiento de un barco a otro muelle en un periodo.

    Returns
    -------
    objetivo : LpAffineExpression
        Expresión lineal que representa la función objetivo a maximizar.
    """

    objetivo = pulp.lpSum(x[p_k,d,loc]*proyectos.loc[periodos.loc[p_k,'proyecto_id'], 'facturacion_diaria'] for p_k, d, loc in x.keys()) - MOVED_PROJECTS_PENALTY_PER_MOVEMENT*pulp.lpSum(m.values())
    return objetivo

In [18]:
# Definir restricciones

def definir_restricciones(x: dict, y: dict, m: dict, dias: list, periodos: pd.DataFrame, muelles: pd.DataFrame, proyectos: pd.DataFrame, set_a_optimizar: set, set_no_optimizar: set) -> dict:
    """Define las restricciones del problema de optimización.

    Parameters
    ----------
    x : dict
        Diccionario de variables binarias que indican si un periodo está asignado a un muelle en un día específico.
    y : dict
        Diccionario de variables binarias que indican si un periodo está asignado (1) o no (0).
    m : dict
        Diccionario de variables binarias que indican si un periodo se mueve de un muelle a otro en un día específico.
    dias : list
        Lista de días desde la fecha inicial hasta la fecha final de los periodos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    muelles : pd.DataFrame
        DataFrame con las dimensiones de los muelles.
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    set_a_optimizar : set
        Set de proyectos a optimizar.
    set_no_optimizar : set
        Set de proyectos que no optimizar.
    
    Returns
    -------
    restricciones : dict
        Diccionario de restricciones del problema de optimización.
    """

    # Crear diccionario de longitud total de barcos confirmados por ubicación y por dia
    periodos['eslora'] = periodos['proyecto_id'].map(proyectos['eslora'])
    longitudes_confirmados = periodos[periodos['proyecto_id'].isin(set_no_optimizar)].explode('dias').groupby(['dias', 'nombre_area'])['eslora'].sum().to_dict()

    restricciones = {}

    # Cada día del periodo debe estar asignado exactamente a un muelle si y[p] = 1 y a ninguno si y[p] = 0
    restricciones.update(
        {
            f"Asignacion_{p_k}_{d}": (pulp.lpSum(x[(p_k, d, loc)] for loc in periodos.loc[p_k, 'ubicaciones']) == y[p], f"Asignacion{p_k}_{d}")
            for p in proyectos[proyectos['proyecto_a_optimizar']].index
            for p_k in periodos[periodos["proyecto_id"] == p].index
            for d in periodos.loc[p_k, 'dias']
        }
    )

    # Los barcos en el mismo muelle no pueden exceder la longitud del muelle
    restricciones.update(
        {
            f"Longitud_Muelle_{d}_{loc}": (pulp.lpSum(x.get((p, d, loc),0) * proyectos.loc[periodos.loc[p, 'proyecto_id'], 'eslora'] for p in periodos.index) + longitudes_confirmados.get((d,loc),0) <= muelles.loc[loc, 'longitud'], 
            f"Longitud_Muelle_{d}_{loc}")
            for loc in muelles.index
            for d in dias
        }
    )

    # m es igual a uno para un periodo en un día d si en d-1 está en un muelle diferente, si no es cero
    restricciones.update(
        {
            f"Movimiento_{p_k}_{d}_{loc}_mayor": (m[(p_k, d)] >= x[(p_k, d, loc)] - x[(p_k, d-1, loc)],
            f"Movimiento_{p_k}_{d}_{loc}_mayor")
            for p_k in periodos[periodos["proyecto_id"].isin(set_a_optimizar)].index if len(periodos.loc[p_k, 'ubicaciones']) > 1
            for d in periodos.loc[p_k, 'dias'][1:]  # Comenzar desde el segundo día para evitar d-1 fuera de rango
            for loc in periodos.loc[p_k, 'ubicaciones']
        }
    )

    restricciones.update(
        {
            f"Movimiento_{p_k}_{d}_{loc}_menor": (m[(p_k, d)] <= 2 - x[(p_k, d, loc)] - x[(p_k, d-1, loc)],
            f"Movimiento_{p_k}_{d}_{loc}_menor")
            for p_k in periodos[periodos["proyecto_id"].isin(set_a_optimizar)].index if len(periodos.loc[p_k, 'ubicaciones']) > 1
            for d in periodos.loc[p_k, 'dias'][1:]  # Comenzar desde el segundo día para evitar d-1 fuera de rango
            for loc in periodos.loc[p_k, 'ubicaciones']
        }
    )

    return restricciones

In [19]:
# Resolver el problema de optimización

def resolver_problema(objetivo: pulp.LpAffineExpression, restricciones: dict) -> pulp.LpProblem:
    """Resuelve el problema de optimización utilizando PuLP.

    Parameters
    ----------
    objetivo : LpAffineExpression
        Expresión lineal que representa la función objetivo a maximizar.
    restricciones : dict
        Diccionario de restricciones del problema de optimización.
   
    Returns
    -------
    prob : LpProblem
        Objeto LpProblem que representa el problema de optimización.
    """

    prob = pulp.LpProblem("Asignación de Periodos a Muelles", pulp.LpMaximize)
    prob += objetivo

    for c in restricciones.values():
            prob += c

    prob.solve()

    return prob

In [20]:
# Imprimir asignación de proyectos por muelle

def imprimir_asignacion(prob, x, dias, periodos, muelles):
    
    print("Estado de la solución:", pulp.LpStatus[prob.status])
    print("\nAsignación de Proyectos a Muelles:\n")
    print("Día\t", "\t".join(muelles.index))

    for d in dias:
        row = f"{d}\t "
        for m in muelles.index:
            for p in periodos.index:
                if (p,d,m) in x.keys():
                    if x[(p,d,m)].varValue == 1:
                        row += f"{p}\t\t"
                        break
                elif periodos.loc[p, 'nombre_area'] == m and d in periodos.loc[p, 'dias']:
                    row += f"{p}\t\t"
                    break
            else:
                row += "N/A\t\t"
        print(row)


In [131]:
# Dataframe de resultados

def crear_dataframe_resultados(x: dict, periodos: pd.DataFrame, set_a_optimizar: set, fecha_inicial: pd.Timestamp) -> pd.DataFrame:
    """Crea un DataFrame con los resultados de la asignación de periodos a muelles.

    Parameters
    ----------
    x : dict
        Diccionario de variables binarias que indican si un periodo está asignado a un muelle en un día específico.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    set_a_optimizar : set
        Set de proyectos a optimizar.
    fecha_inicial : pd.Timestamp
        Fecha inicial del primer periodo de los proyectos.
    
    Returns
    -------
    resultados : pd.DataFrame
        DataFrame con la asignación de periodos a muelles, incluyendo proyecto_id, periodo_id, ubicación, fecha_inicio y fecha_fin.
    """
    
    data = {
        'proyecto_id': [],
        'periodo_id': [],
        'id_proyecto_reparacion': [],
        'ubicacion': [],
        'dia': [],
        }

    for p_k, d, loc in x.keys():
        if x[(p_k, d, loc)].varValue == 1:
            data['proyecto_id'].append(periodos.loc[p_k, 'proyecto_id'])
            data['periodo_id'].append(periodos.loc[p_k, 'periodo_id'])
            data['id_proyecto_reparacion'].append(p_k)
            data['ubicacion'].append(loc)
            data['dia'].append(d)

    resultados = pd.DataFrame(data).sort_values(by=['proyecto_id','periodo_id','ubicacion','dia']).groupby(['proyecto_id','periodo_id','id_proyecto_reparacion','ubicacion']).agg(fecha_inicio = ('dia', 'min'), fecha_fin = ('dia', 'max')).reset_index()

    for p_k in periodos[periodos["proyecto_id"].isin(set_a_optimizar)].index:
        if p_k not in list(resultados['id_proyecto_reparacion']):
            resultados.loc[len(resultados)] = {
            'proyecto_id': periodos.loc[p_k, 'proyecto_id'],
            'periodo_id': periodos.loc[p_k, 'periodo_id'],
            'id_proyecto_reparacion': p_k,
            'ubicacion': periodos.loc[p_k, 'nombre_area'],
            'fecha_inicio': periodos.loc[p_k, 'fecha_inicio'],
            'fecha_fin': periodos.loc[p_k, 'fecha_fin']
            }

    resultados['fecha_inicio'] = pd.to_datetime(resultados['fecha_inicio'], unit="D", origin = fecha_inicial).dt.date
    resultados['fecha_fin'] = pd.to_datetime(resultados['fecha_fin'], unit="D", origin = fecha_inicial).dt.date
    resultados['id_resultado'] = resultados.apply(lambda row: f"{row['proyecto_id']}_{row['fecha_inicio']}_{row['fecha_fin']}_{row['ubicacion']}", axis=1)
    resultados = resultados.sort_values(by=['proyecto_id','periodo_id'], ignore_index = True)

    return resultados


In [132]:
# Optimize

def optimize():
    proyectos, periodos, muelles, fecha_inicial, MOVED_PROJECTS_PENALTY_PER_MOVEMENT = leer_datos()
    proyectos, periodos, muelles, dias, set_a_optimizar, set_no_optimizar = preprocesar_datos(proyectos, periodos, muelles, fecha_inicial)
    
    x, y, m = definir_variables(periodos, set_a_optimizar)
    objetivo = definir_funcion_objetivo(x, m, proyectos, periodos, set_a_optimizar, MOVED_PROJECTS_PENALTY_PER_MOVEMENT)
    restricciones = definir_restricciones(x, y, m, dias, periodos, muelles, proyectos, set_a_optimizar, set_no_optimizar)
    
    prob = resolver_problema(objetivo, restricciones)
    resultados = crear_dataframe_resultados(x, periodos, set_a_optimizar, fecha_inicial)
    imprimir_asignacion(prob, x, dias, periodos, muelles)
    print("\nResultados:\n\n", resultados)


In [133]:
optimize()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/x4/4ffrr56n3jzb55xn5fzcplwc0000gn/T/6cfc4d1f2f2645e0a731faaf6b3b2714-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/x4/4ffrr56n3jzb55xn5fzcplwc0000gn/T/6cfc4d1f2f2645e0a731faaf6b3b2714-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 104 COLUMNS
At line 459 RHS
At line 559 BOUNDS
At line 611 ENDATA
Problem MODEL has 99 rows, 51 columns and 203 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9583.33 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0004I processed model has 31 rows, 21 columns (21 integer (21 of which binary)) and 105 elements
Cutoff increment increased from 1e-05 to 49.9999
Cbc0038I Ini

/opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
